<a href="https://colab.research.google.com/github/succSeeded/dl-2025/blob/main/hws/week03_convnets/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2.2: The Quest For A Better Network

In this assignment you will build a monster network to solve CIFAR10 image classification.

This notebook is intended as a sequel to seminar 3, please give it a try if you haven't done so yet.

* The ultimate quest is to create a network that has as high __accuracy__ as you can push it.
* There is a __mini-report__ at the end that you will have to fill in. We recommend reading it first and filling it while you iterate.

## Grading
* starting at zero points
* +20% for describing your iteration path in a report below.
* +20% for building a network that gets above 20% accuracy
* +10% for beating each of these milestones on __TEST__ dataset:
    * 50% (50% points)
    * 60% (60% points)
    * 65% (70% points)
    * 70% (80% points)
    * 75% (90% points)
    * 80% (full points)
    
## Restrictions
* Please do NOT use pre-trained networks for this assignment until you reach 80%.
 * In other words, base milestones must be beaten without pre-trained nets (and such net must be present in the e-mail). After that, you can use whatever you want.
* you __can__ use validation data for training, but you __can't__ do anything with test data apart from running the evaluation procedure.

## Tips on what can be done:


 * __Network size__
   * MOAR neurons,
   * MOAR layers, ([torch.nn docs](http://pytorch.org/docs/master/nn.html))

   * Nonlinearities in the hidden layers
     * tanh, relu, leaky relu, etc
   * Larger networks may take more epochs to train, so don't discard your net just because it could didn't beat the baseline in 5 epochs.

   * Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fhtagn!


### The main rule of prototyping: one change at a time
   * By now you probably have several ideas on what to change. By all means, try them out! But there's a catch: __never test several new things at once__.


### Optimization
   * Training for 100 epochs regardless of anything is probably a bad idea.
   * Some networks converge over 5 epochs, others - over 500.
   * Way to go: stop when validation score is 10 iterations past maximum
   * You should certainly use adaptive optimizers
     * rmsprop, nesterov_momentum, adam, adagrad and so on.
     * Converge faster and sometimes reach better optima
     * It might make sense to tweak learning rate/momentum, other learning parameters, batch size and number of epochs
   * __BatchNormalization__ (nn.BatchNorm2d) for the win!
     * Sometimes more batch normalization is better.
   * __Regularize__ to prevent overfitting
     * Add some L2 weight norm to the loss function, PyTorch will do the rest
       * Can be done manually or with weight_decay parameter of a optimizer ([for example SGD's doc](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD)).
     * Dropout (`nn.Dropout`) - to prevent overfitting
       * Don't overdo it. Check if it actually makes your network better
   
### Convolution architectures
   * This task __can__ be solved by a sequence of convolutions and poolings with batch_norm and ReLU seasoning, but you shouldn't necessarily stop there.
   * [Inception family](https://hacktilldawn.com/2016/09/25/inception-modules-explained-and-implemented/), [ResNet family](https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035?gi=9018057983ca), [Densely-connected convolutions (exotic)](https://arxiv.org/abs/1608.06993), [Capsule networks (exotic)](https://arxiv.org/abs/1710.09829)
   * Please do try a few simple architectures before you go for resnet-152.
   * Warning! Training convolutional networks can take long without GPU. That's okay.
     * If you are CPU-only, we still recomment that you try a simple convolutional architecture
     * a perfect option is if you can set it up to run at nighttime and check it up at the morning.
     * Make reasonable layer size estimates. A 128-neuron first convolution is likely an overkill.
     * __To reduce computation__ time by a factor in exchange for some accuracy drop, try using __stride__ parameter. A stride=2 convolution should take roughly 1/4 of the default (stride=1) one.

   
### Data augmemntation
   * getting 5x as large dataset for free is a great
     * Zoom-in+slice = move
     * Rotate+zoom(to remove black stripes)
     * Add Noize (gaussian or bernoulli)
   * Simple way to do that (if you have PIL/Image):
     * ```from scipy.misc import imrotate,imresize```
     * and a few slicing
     * Other cool libraries: cv2, skimake, PIL/Pillow
   * A more advanced way is to use torchvision transforms:
```
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    trainset = torchvision.datasets.CIFAR10(
        root=path_to_cifar_like_in_seminar,
        train=True,
        download=True,
        transform=transform_train
    )
    trainloader = torch.utils.data.DataLoader(
        trainset,
        batch_size=128,
        shuffle=True,
        num_workers=2
    )
```
   * Or use this tool from Keras (requires theano/tensorflow): [tutorial](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html), [docs](https://keras.io/preprocessing/image/)
   * Stay realistic. There's usually no point in flipping dogs upside down as that is not the way you usually see them.

In [8]:
import time
import pathlib

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split


def train_val_split(dataset, val_size=0.2):
    """
    Split torch datasets into train and validation parts

    Args:
        dataset: incoming torch.Dataset object
        val_size: portion of the dataset that will be used for validation (default: 0.2)
    """
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_size)
    return (Subset(dataset, train_idx), Subset(dataset, val_idx))


device = "cuda" if torch.cuda.is_available() else "cpu"

means = np.array((0.4914, 0.4822, 0.4465))  # statistics from dataset documentation
stds = np.array((0.2023, 0.1994, 0.2010))

transform_augment = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation([-30, 30]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(means, stds),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(means, stds),
])


test_dataset = CIFAR10("./data/", train=False, download=True, transform=transform_test)
train_val_dataset = CIFAR10("./data/", train=True, download=True, transform=transform_augment)

train_dataset, val_dataset = train_val_split(train_val_dataset)

In [12]:
def train(model, opt, train_dataset, val_dataset, num_epochs:int = 100, batch_size:int = 64, stop:int = 7, device:str = None):
    """
    A function for training torch models that I kindly took from the professor (sorry) but now with torch dataloaders

    Args:
        model: torch model to train
        opt: optimizer for that model
        train_dataset: torch.Dataset used for training
        val_dataset: torch.Dataset used for validation
        num_epochs: total amount of full passes over training data (default: 100)
        batch_size: number of samples processed in one SGD iteration (default: 64)
        stop: number of iterations that loss can decrease for before the training proces stops (default: 7)
    """
    train_loss = []
    val_accuracy = []
    best_val_acc = 0.0
    best_epoch = 0

    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    for epoch in range(num_epochs):
        # In each epoch, we do a full pass over the training data:
        start_time = time.time()
        model.train(True) # enable dropout / batch_norm training behavior
        for (X_batch, y_batch) in train_dataloader:
            X_batch = X_batch.to(torch.float32).to(device)
            y_batch = y_batch.to(torch.int64).to(device)
            # train on batch
            logits = model(X_batch)
            loss = F.cross_entropy(logits, y_batch).mean()
            loss.backward()
            opt.step()
            opt.zero_grad()
            train_loss.append(loss.item())  # .item() = convert 1-value Tensor to float

        # And a full pass over the validation data:
        model.train(False)     # disable dropout / use averages for batch_norm
        with torch.no_grad():  # do not store intermediate activations
            for (X_batch, y_batch) in val_dataloader:
                X_batch = X_batch.to(torch.float32).to(device)
                y_batch = y_batch.detach().cpu().numpy()
                logits = model(X_batch)
                y_pred = logits.argmax(-1).detach().cpu().numpy()
                val_accuracy.append(np.mean(y_batch == y_pred))

        mean_val_acc = np.mean(val_accuracy[-len(val_dataset) // batch_size :])

        if best_val_acc < mean_val_acc:
            best_val_acc = mean_val_acc
            best_epoch = epoch
            pathlib.Path("./models/").mkdir(exist_ok=True)
            torch.save(model.state_dict(), f"models/best_model.pt2")


        if epoch - best_epoch > stop:
            print("Model did not see any loss improvements for %i epochs, aborting..." % (stop))
            model.load_state_dict(torch.load(f"models/best_model.pt2", weights_only=True))
            model.eval()
            break

        # Then we print the results for this epoch:
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss (in-iteration): \t{:.6f}".format(
            np.mean(train_loss[-len(train_dataset) // batch_size :])))
        print("  validation accuracy: \t\t\t{:.2f} %".format(
            mean_val_acc * 100))

    print(f"Finished training. Best validation accuracy: {best_val_acc*100:.2f} %")

In [10]:
def evaluate(model, test_dataset):
    """
    A function for evaluating torch models but now with torch dataloaders

    Args:
        model: torch model to evaluate
        test_dataset: torch.Dataset that contains test data
    """
    model.train(False) # disable dropout / use averages for batch_norm
    test_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size=500, shuffle=True, num_workers=1)
    test_batch_acc = []

    for X_batch, y_batch in test_dataloader:
        X_batch = X_batch.to(torch.float32).to(device)
        y_batch = y_batch.detach().cpu().numpy()
        logits = model(X_batch)
        y_pred = logits.max(1)[1].data.cpu().numpy()
        test_batch_acc.append(np.mean(y_batch == y_pred))

    test_accuracy = np.mean(test_batch_acc)

    print("Final results:")
    print("  test accuracy:\t\t{:.2f} %".format(
        test_accuracy * 100))

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, is_last=False):
        super(BasicBlock, self).__init__()
        self.is_last = is_last
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        preact = out
        out = F.relu(out)
        if self.is_last:
            return out, preact
        else:
            return out

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, is_last=False):
        super(Bottleneck, self).__init__()
        self.is_last = is_last
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        preact = out
        out = F.relu(out)
        if self.is_last:
            return out, preact
        else:
            return out

In [19]:
# model = nn.Sequential(
#     nn.Conv2d(3, 10, kernel_size=(3,3)),  # [10, 30, 30]
#     nn.GELU(),
#     nn.MaxPool2d(2), # [10, 15, 15]
#     nn.Flatten(),
#     nn.Linear(10 * 15 * 15, 100),
#     nn.GELU(),
#     nn.Dropout(p=0.1),
#     nn.Linear(100, 10)
# ).to(device) # this produces:
#                     58.42% accuracy on test set, ~7.1s per epoch with GELU
#                     55.48% accuracy on test set, ~6.9s per epoch with tanh
#                     54.55% accuracy on test set, ~6.8s per epoch with ReLU

# # This is meant to mirror AlexNet architecture
# model = nn.Sequential(
#     nn.Conv2d(3, 16, kernel_size=(5,5)),  # [16, 28, 28]
#     nn.ReLU(),
#     nn.Conv2d(16, 64, kernel_size=(3,3)), # [64, 26, 26]
#     nn.MaxPool2d(2), # [64, 13, 13]
#     nn.Conv2d(64, 96, kernel_size=(3,3), padding=1), # [96, 13, 13]
#     nn.Conv2d(96, 96, kernel_size=(3,3), padding=1), # [96, 13, 13]
#     nn.Conv2d(96, 64, kernel_size=(3,3), padding=1), # [96, 13, 13]
#     nn.MaxPool2d(2), # [64, 6, 6]
#     nn.Flatten(),
#     nn.Linear(64 * 6 * 6, 4096),
#     nn.ReLU(),
#     nn.Linear(4096, 4096),
#     nn.ReLU(),
#     nn.Dropout(p=0.1),
#     nn.Linear(4096, 10)
# ).to(device) # this produces:
# #                   71.36% accuracy on test set, ~12.5s per epoch with ReLU

# This is meant to replicate ResNet-18 architecture
model = nn.Sequential(
    
).to(device)

opt = torch.optim.Adam(model.parameters())

train(model, opt, train_dataset, val_dataset, device=device, stop=10)
evaluate(model, test_dataset)

Epoch 1 of 100 took 12.485s
  training loss (in-iteration): 	1.795707
  validation accuracy: 			41.82 %
Epoch 2 of 100 took 12.483s
  training loss (in-iteration): 	1.610396
  validation accuracy: 			43.86 %
Epoch 3 of 100 took 12.459s
  training loss (in-iteration): 	1.518357
  validation accuracy: 			48.40 %
Epoch 4 of 100 took 12.458s
  training loss (in-iteration): 	1.425869
  validation accuracy: 			51.10 %
Epoch 5 of 100 took 12.455s
  training loss (in-iteration): 	1.350977
  validation accuracy: 			52.45 %
Epoch 6 of 100 took 12.525s
  training loss (in-iteration): 	1.308229
  validation accuracy: 			54.66 %
Epoch 7 of 100 took 12.306s
  training loss (in-iteration): 	5.398445
  validation accuracy: 			10.27 %
Epoch 8 of 100 took 12.289s
  training loss (in-iteration): 	2.302742
  validation accuracy: 			9.57 %
Epoch 9 of 100 took 12.297s
  training loss (in-iteration): 	2.302716
  validation accuracy: 			9.83 %
Epoch 10 of 100 took 12.305s
  training loss (in-iteration): 	2.30

KeyboardInterrupt: 